# Cálculo score Junglers
  
En el presente archivo se realizará la comparativa entre jugadores profesionales y los jugadores que tienen potencial para serlo.

In [ ]:
# Carga de librerías y recursos

In [50]:
import numpy as np
import pandas as pd
import time

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder

from scipy.spatial.distance import pdist, squareform

In [3]:
rookies = pd.read_csv('../data/total.csv')
profesional = pd.read_csv('../data/total_prof.csv')


In [4]:
rookies.head()

,nombre_invocador,region,liga,lps,cola,ranking_mundial,ranking_porcentaje,wins,loses,n_games,winrate_total,liga_label,kills_player_kda,deaths_player_kda,assists_player_kda,rol_1,games_per_rol_1,winrate_per_rol_1,rol_2,games_per_rol_2,winrate_per_rol_2,rol_3,games_per_rol_3,winrate_per_rol_3,champ_1,regional_rank_1,kills_champ_1,deaths_champ_1,assists_champ_1,games_champ_1,winrate_champ_1,champ_2,regional_rank_2,kills_champ_2,deaths_champ_2,assists_champ_2,games_champ_2,winrate_champ_2,champ_3,regional_rank_3,kills_champ_3,deaths_champ_3,assists_champ_3,games_champ_3,winrate_champ_3,champ_4,regional_rank_4,kills_champ_4,deaths_champ_4,assists_champ_4,games_champ_4,winrate_champ_4,champ_5,regional_rank_5,kills_champ_5,deaths_champ_5,assists_champ_5,games_champ_5,winrate_champ_5,champ_6,regional_rank_6,kills_champ_6,deaths_champ_6,assists_champ_6,games_champ_6,winrate_champ_6,champ_7,regional_rank_7,kills_champ_7,deaths_champ_7,assists_champ_7,games_champ_7,winrate_champ_7,champ_8,regional_rank_8,kills_champ_8,deaths_champ_8,assists_champ_8,games_champ_8,winrate_champ_8,nivel
0,Agurin,EUW,Challenger,1742,Soloqueue,4,0.000043,242,174,436,57.6,2,6.4,4.6,8.5,Jungler,416,58.2,Top,13.0,46.2,Mid,7.0,57.1,Jarvan IV,2,6.2,5.0,12.6,116,60.3,Elise,3641,7.2,3.8,6.7,100,67.0,Wukong,42,5.9,4.3,7.0,68,55.9,Kha'Zix,3,6.3,5.1,7.8,51,51.0,Evelynn,69,8.2,4.3,6.4,33,54.5,Diana,85,6.8,5.2,6.5,19,47.4,Karma,384,2.7,5.2,11.5,13,53.8,Nocturne,2894,5.3,5.7,6.9,9,33.3,rookie
1,6K OnFleek,NaN,Challenger,1859,Soloqueue,1,0.000010,150,111,260,57.7,2,6.3,4.7,7.3,Jungler,205,57.1,Top,56.0,58.9,Mid,7.0,28.6,Sylas,1,9.1,4.8,6.5,44,50.0,Jax,137,5.5,4.5,3.2,32,65.6,Vi,148,6.2,5.1,8.9,30,63.3,Lee Sin,100,6.3,4.4,7.6,24,58.3,Elise,332,6.9,4.4,8.9,16,68.8,Viego,463,6.3,4.7,7.4,16,68.8,Maokai,255,4.6,4.1,9.9,16,56.3,Kindred,1853,6.3,6.2,8.6,13,53.8,rookie
2,frosty,BR,Challenger,1760,Soloqueue,3,0.000032,194,141,336,57.7,2,5.1,5.0,12.4,Support,249,57.4,AD Carry,78.0,60.3,Mid,6.0,66.7,Ashe,1,3.6,4.9,13.5,68,52.9,Karma,21,4.2,4.5,16.8,34,64.7,Heimerdinger,70,4.6,5.4,11.6,34,58.8,Rakan,37,2.0,4.1,17.4,33,60.6,Ezreal,86,8.6,5.3,7.2,29,55.2,Zeri,359,9.3,4.4,6.8,23,73.9,Nautilus,164,2.2,4.8,14.8,17,58.8,Varus,284,7.9,5.3,9.5,16,56.3,rookie
3,Noah7,EUW,Challenger,1782,Soloqueue,2,0.000021,243,159,402,60.7,2,9.0,5.5,7.8,AD Carry,336,61.6,Jungler,70.0,52.9,Support,5.0,100.0,Zeri,1,10.0,5.5,7.5,103,66.0,Lucian,17,10.4,6.1,7.9,54,66.7,Ezreal,2,8.4,5.0,6.6,49,59.2,Graves,54,8.9,4.4,7.9,47,59.6,Caitlyn,28,6.8,5.8,6.3,30,53.3,Varus,4486,9.0,6.4,8.7,26,53.8,Kai'Sa,1646,10.2,5.7,6.6,22,68.2,Xayah,1354,10.0,5.3,8.8,17,58.8,rookie
4,random gap,BR,Challenger,1056,Soloqueue,315,0.003400,113,89,213,55.9,2,5.9,4.2,5.9,Mid,105,53.3,Top,101.0,59.4,Jungler,3.0,66.7,K'Sante,85,5.3,3.6,7.0,98,59.2,Akshan,3745,8.9,5.1,5.2,29,55.2,Tristana,7,6.7,4.4,4.1,11,45.5,Jax,121,5.5,5.9,3.6,10,60.0,Zeri,1726,7.0,3.8,3.3,9,66.7,Ryze,2081,3.0,2.9,2.3,9,44.4,Irelia,0,5.9,6.3,4.9,7,42.9,Kassadin,0,4.5,3.2,1.7,6,83.3,rookie


In [10]:
profesional.head()

,nombre_invocador,region,liga,lps,cola,ranking_mundial,ranking_porcentaje,wins,loses,n_games,winrate_total,liga_label,kills_player_kda,deaths_player_kda,assists_player_kda,rol_1,games_per_rol_1,winrate_per_rol_1,rol_2,games_per_rol_2,winrate_per_rol_2,rol_3,games_per_rol_3,winrate_per_rol_3,champ_1,regional_rank_1,kills_champ_1,deaths_champ_1,assists_champ_1,games_champ_1,winrate_champ_1,champ_2,regional_rank_2,kills_champ_2,deaths_champ_2,assists_champ_2,games_champ_2,winrate_champ_2,champ_3,regional_rank_3,kills_champ_3,deaths_champ_3,assists_champ_3,games_champ_3,winrate_champ_3,champ_4,regional_rank_4,kills_champ_4,deaths_champ_4,assists_champ_4,games_champ_4,winrate_champ_4,champ_5,regional_rank_5,kills_champ_5,deaths_champ_5,assists_champ_5,games_champ_5,winrate_champ_5,champ_6,regional_rank_6,kills_champ_6,deaths_champ_6,assists_champ_6,games_champ_6,winrate_champ_6,champ_7,regional_rank_7,kills_champ_7,deaths_champ_7,assists_champ_7,games_champ_7,winrate_champ_7,champ_8,regional_rank_8,kills_champ_8,deaths_champ_8,assists_champ_8,games_champ_8,winrate_champ_8,nivel
0,Hide on bush,KR,GrandMaster,542,Soloqueue,4736,0.0490,149,135,284,52.5,1,5.4,4.2,6.4,Mid,246,54.1,Jungler,25.0,44.0,Top,8.0,37.5,Annie,514,5.0,2.8,6.7,28,67.9,Aurelion Sol,245,6.9,3.4,6.6,27,55.6,Yone,850,5.1,4.5,5.4,16,62.5,Sylas,1640,4.4,3.8,5.4,16,50.0,Akali,19442,7.9,3.9,4.3,15,60.0,Ryze,1667,4.9,3.2,6.4,15,53.3,Tristana,5271,5.9,6.7,4.5,13,30.8,Jayce,18688,5.8,6.3,6.6,12,25.0,profesional
1,Jinno Kingdom,EUW,Challenger,801,Soloqueue,1183,0.0120,295,273,568,51.9,2,6.6,4.9,7.5,AD Carry,447,53.7,Jungler,59.0,45.8,Support,37.0,51.4,Varus,299,7.8,4.6,8.0,79,53.2,Zeri,72,6.8,3.4,6.1,64,64.1,Caitlyn,18,6.9,4.8,5.8,51,56.9,Ezreal,97,7.3,5.6,6.6,36,52.8,Sivir,678,6.3,4.4,8.8,32,62.5,Draven,106,7.6,5.4,5.6,29,48.3,Maokai,1391,3.6,4.3,10.4,25,56.0,Xayah,1137,6.9,5.5,5.7,24,37.5,profesional
2,Satoru Gojo03,EUW,Challenger,984,Soloqueue,535,0.0056,77,47,124,62.1,2,7.2,5.2,7.2,AD Carry,120,61.7,Mid,3.0,100.0,Jungler,1.0,0.0,Zeri,122,7.3,4.6,5.9,23,60.9,Draven,880,8.0,5.9,7.7,21,57.1,Varus,1558,6.8,5.8,8.2,17,58.8,Aphelios,1905,6.0,5.4,7.4,17,52.9,Caitlyn,666,5.8,4.5,7.7,13,61.5,Lucian,4463,9.5,6.4,7.7,11,81.8,Xayah,210,8.4,2.8,6.4,9,88.9,Sivir,0,6.3,9.0,11.3,3,66.7,profesional
3,pleroma chronou,EUW,GrandMaster,616,Soloqueue,3944,0.0410,123,107,230,53.5,1,7.1,5.2,6.5,Mid,189,57.7,AD Carry,14.0,28.6,Jungler,13.0,46.2,Aurelion Sol,839,7.5,6.7,7.9,24,54.2,Tristana,1216,8.8,5.0,4.9,22,63.6,Akali,756,9.8,4.0,5.4,17,64.7,Sylas,831,7.5,4.8,4.9,15,53.3,Ryze,3807,7.7,3.8,5.8,12,75.0,Lee Sin,1546,8.4,5.6,4.5,12,66.7,Syndra,0,6.4,5.2,8.8,9,55.6,Gragas,0,2.3,5.9,8.0,7,42.9,profesional
4,Kongenvenderhjem,EUW,Challenger,1203,Soloqueue,156,0.0016,114,73,211,59.7,2,7.1,5.9,5.7,Top,189,60.8,Jungler,7.0,42.9,Mid,7.0,57.1,Jayce,36,6.8,6.1,5.6,38,73.7,Gangplank,10569,7.0,5.7,6.3,28,71.4,Fiora,53,5.0,5.6,4.9,14,50.0,Vladimir,305,4.3,5.0,5.4,11,54.5,Gnar,2672,8.1,4.2,5.3,10,70.0,Quinn,2429,8.8,7.1,6.0,10,60.0,Jax,485,4.2,5.9,4.3,9,33.3,Lee Sin,540,9.3,6.1,4.3,8,62.5,profesional


In [6]:
len(profesional)

38

In [7]:
len(rookies)

3436

In [ ]:
# Se filtran los dfs solo con las columnas numéricas que nos interesan, o en su defecto con aquellas que serán
# etiquetadas y que por lo tanto lo serán.

In [20]:
prof = profesional[['nombre_invocador', 'lps', 'ranking_mundial', 'ranking_porcentaje', 'wins', 'loses', 'n_games', 'winrate_total', 'liga_label', 'kills_player_kda', 'deaths_player_kda', 'assists_player_kda', 'rol_1', 'games_per_rol_1', 'winrate_per_rol_1', 'champ_1', 'regional_rank_1', 'kills_champ_1', 'deaths_champ_1', 'assists_champ_1', 'games_champ_1', 'winrate_champ_1', 'champ_2', 'regional_rank_2', 'kills_champ_2', 'deaths_champ_2', 'assists_champ_2', 'games_champ_2', 'winrate_champ_2', 'champ_3', 'regional_rank_3', 'kills_champ_3', 'deaths_champ_3', 'assists_champ_3', 'games_champ_3', 'winrate_champ_3', 'champ_4', 'regional_rank_4', 'kills_champ_4', 'deaths_champ_4', 'assists_champ_4', 'games_champ_4', 'winrate_champ_4', 'nivel']]

In [12]:
prof.head()

,nombre_invocador,lps,ranking_mundial,ranking_porcentaje,wins,loses,n_games,winrate_total,liga_label,kills_player_kda,deaths_player_kda,assists_player_kda,rol_1,games_per_rol_1,winrate_per_rol_1,champ_1,regional_rank_1,kills_champ_1,deaths_champ_1,assists_champ_1,games_champ_1,winrate_champ_1,champ_2,regional_rank_2,kills_champ_2,deaths_champ_2,assists_champ_2,games_champ_2,winrate_champ_2,champ_3,regional_rank_3,kills_champ_3,deaths_champ_3,assists_champ_3,games_champ_3,winrate_champ_3,champ_4,regional_rank_4,kills_champ_4,deaths_champ_4,assists_champ_4,games_champ_4,winrate_champ_4,champ_5,regional_rank_5,kills_champ_5,deaths_champ_5,assists_champ_5,games_champ_5,winrate_champ_5,nivel
0,Hide on bush,542,4736,0.0490,149,135,284,52.5,1,5.4,4.2,6.4,Mid,246,54.1,Annie,514,5.0,2.8,6.7,28,67.9,Aurelion Sol,245,6.9,3.4,6.6,27,55.6,Yone,850,5.1,4.5,5.4,16,62.5,Sylas,1640,4.4,3.8,5.4,16,50.0,Akali,19442,7.9,3.9,4.3,15,60.0,profesional
1,Jinno Kingdom,801,1183,0.0120,295,273,568,51.9,2,6.6,4.9,7.5,AD Carry,447,53.7,Varus,299,7.8,4.6,8.0,79,53.2,Zeri,72,6.8,3.4,6.1,64,64.1,Caitlyn,18,6.9,4.8,5.8,51,56.9,Ezreal,97,7.3,5.6,6.6,36,52.8,Sivir,678,6.3,4.4,8.8,32,62.5,profesional
2,Satoru Gojo03,984,535,0.0056,77,47,124,62.1,2,7.2,5.2,7.2,AD Carry,120,61.7,Zeri,122,7.3,4.6,5.9,23,60.9,Draven,880,8.0,5.9,7.7,21,57.1,Varus,1558,6.8,5.8,8.2,17,58.8,Aphelios,1905,6.0,5.4,7.4,17,52.9,Caitlyn,666,5.8,4.5,7.7,13,61.5,profesional
3,pleroma chronou,616,3944,0.0410,123,107,230,53.5,1,7.1,5.2,6.5,Mid,189,57.7,Aurelion Sol,839,7.5,6.7,7.9,24,54.2,Tristana,1216,8.8,5.0,4.9,22,63.6,Akali,756,9.8,4.0,5.4,17,64.7,Sylas,831,7.5,4.8,4.9,15,53.3,Ryze,3807,7.7,3.8,5.8,12,75.0,profesional
4,Kongenvenderhjem,1203,156,0.0016,114,73,211,59.7,2,7.1,5.9,5.7,Top,189,60.8,Jayce,36,6.8,6.1,5.6,38,73.7,Gangplank,10569,7.0,5.7,6.3,28,71.4,Fiora,53,5.0,5.6,4.9,14,50.0,Vladimir,305,4.3,5.0,5.4,11,54.5,Gnar,2672,8.1,4.2,5.3,10,70.0,profesional


In [18]:
rook = rookies[['nombre_invocador', 'lps', 'ranking_mundial', 'ranking_porcentaje', 'wins', 'loses', 'n_games', 'winrate_total', 'liga_label', 'kills_player_kda', 'deaths_player_kda', 'assists_player_kda', 'rol_1', 'games_per_rol_1', 'winrate_per_rol_1', 'champ_1', 'regional_rank_1', 'kills_champ_1', 'deaths_champ_1', 'assists_champ_1', 'games_champ_1', 'winrate_champ_1', 'champ_2', 'regional_rank_2', 'kills_champ_2', 'deaths_champ_2', 'assists_champ_2', 'games_champ_2', 'winrate_champ_2', 'champ_3', 'regional_rank_3', 'kills_champ_3', 'deaths_champ_3', 'assists_champ_3', 'games_champ_3', 'winrate_champ_3', 'champ_4', 'regional_rank_4', 'kills_champ_4', 'deaths_champ_4', 'assists_champ_4', 'games_champ_4', 'winrate_champ_4', 'nivel']]

In [19]:
rook.head()

,nombre_invocador,lps,ranking_mundial,ranking_porcentaje,wins,loses,n_games,winrate_total,liga_label,kills_player_kda,deaths_player_kda,assists_player_kda,rol_1,games_per_rol_1,winrate_per_rol_1,champ_1,regional_rank_1,kills_champ_1,deaths_champ_1,assists_champ_1,games_champ_1,winrate_champ_1,champ_2,regional_rank_2,kills_champ_2,deaths_champ_2,assists_champ_2,games_champ_2,winrate_champ_2,champ_3,regional_rank_3,kills_champ_3,deaths_champ_3,assists_champ_3,games_champ_3,winrate_champ_3,champ_4,regional_rank_4,kills_champ_4,deaths_champ_4,assists_champ_4,games_champ_4,winrate_champ_4,nivel
0,Agurin,1742,4,0.000043,242,174,436,57.6,2,6.4,4.6,8.5,Jungler,416,58.2,Jarvan IV,2,6.2,5.0,12.6,116,60.3,Elise,3641,7.2,3.8,6.7,100,67.0,Wukong,42,5.9,4.3,7.0,68,55.9,Kha'Zix,3,6.3,5.1,7.8,51,51.0,rookie
1,6K OnFleek,1859,1,0.000010,150,111,260,57.7,2,6.3,4.7,7.3,Jungler,205,57.1,Sylas,1,9.1,4.8,6.5,44,50.0,Jax,137,5.5,4.5,3.2,32,65.6,Vi,148,6.2,5.1,8.9,30,63.3,Lee Sin,100,6.3,4.4,7.6,24,58.3,rookie
2,frosty,1760,3,0.000032,194,141,336,57.7,2,5.1,5.0,12.4,Support,249,57.4,Ashe,1,3.6,4.9,13.5,68,52.9,Karma,21,4.2,4.5,16.8,34,64.7,Heimerdinger,70,4.6,5.4,11.6,34,58.8,Rakan,37,2.0,4.1,17.4,33,60.6,rookie
3,Noah7,1782,2,0.000021,243,159,402,60.7,2,9.0,5.5,7.8,AD Carry,336,61.6,Zeri,1,10.0,5.5,7.5,103,66.0,Lucian,17,10.4,6.1,7.9,54,66.7,Ezreal,2,8.4,5.0,6.6,49,59.2,Graves,54,8.9,4.4,7.9,47,59.6,rookie
4,random gap,1056,315,0.003400,113,89,213,55.9,2,5.9,4.2,5.9,Mid,105,53.3,K'Sante,85,5.3,3.6,7.0,98,59.2,Akshan,3745,8.9,5.1,5.2,29,55.2,Tristana,7,6.7,4.4,4.1,11,45.5,Jax,121,5.5,5.9,3.6,10,60.0,rookie


In [144]:
rook.shape

(3436, 44)

In [ ]:
# Mediante la siguiente línea de código se eliminan aquellos registros de jugadores profesionales que aparecen en
# el df de rookies.

In [145]:
rook = rook[~rook['nombre_invocador'].isin(prof['nombre_invocador'].unique())]

In [146]:
rook.shape

(3405, 44)

In [ ]:
# Concatenamos ambos dfs

In [147]:
data = pd.concat([rook, prof])

In [148]:
data.head()

,nombre_invocador,lps,ranking_mundial,ranking_porcentaje,wins,loses,n_games,winrate_total,liga_label,kills_player_kda,deaths_player_kda,assists_player_kda,rol_1,games_per_rol_1,winrate_per_rol_1,champ_1,regional_rank_1,kills_champ_1,deaths_champ_1,assists_champ_1,games_champ_1,winrate_champ_1,champ_2,regional_rank_2,kills_champ_2,deaths_champ_2,assists_champ_2,games_champ_2,winrate_champ_2,champ_3,regional_rank_3,kills_champ_3,deaths_champ_3,assists_champ_3,games_champ_3,winrate_champ_3,champ_4,regional_rank_4,kills_champ_4,deaths_champ_4,assists_champ_4,games_champ_4,winrate_champ_4,nivel
0,Agurin,1742,4,0.000043,242,174,436,57.6,2,6.4,4.6,8.5,Jungler,416,58.2,Jarvan IV,2,6.2,5.0,12.6,116,60.3,Elise,3641,7.2,3.8,6.7,100,67.0,Wukong,42,5.9,4.3,7.0,68,55.9,Kha'Zix,3,6.3,5.1,7.8,51,51.0,rookie
1,6K OnFleek,1859,1,0.000010,150,111,260,57.7,2,6.3,4.7,7.3,Jungler,205,57.1,Sylas,1,9.1,4.8,6.5,44,50.0,Jax,137,5.5,4.5,3.2,32,65.6,Vi,148,6.2,5.1,8.9,30,63.3,Lee Sin,100,6.3,4.4,7.6,24,58.3,rookie
2,frosty,1760,3,0.000032,194,141,336,57.7,2,5.1,5.0,12.4,Support,249,57.4,Ashe,1,3.6,4.9,13.5,68,52.9,Karma,21,4.2,4.5,16.8,34,64.7,Heimerdinger,70,4.6,5.4,11.6,34,58.8,Rakan,37,2.0,4.1,17.4,33,60.6,rookie
3,Noah7,1782,2,0.000021,243,159,402,60.7,2,9.0,5.5,7.8,AD Carry,336,61.6,Zeri,1,10.0,5.5,7.5,103,66.0,Lucian,17,10.4,6.1,7.9,54,66.7,Ezreal,2,8.4,5.0,6.6,49,59.2,Graves,54,8.9,4.4,7.9,47,59.6,rookie
4,random gap,1056,315,0.003400,113,89,213,55.9,2,5.9,4.2,5.9,Mid,105,53.3,K'Sante,85,5.3,3.6,7.0,98,59.2,Akshan,3745,8.9,5.1,5.2,29,55.2,Tristana,7,6.7,4.4,4.1,11,45.5,Jax,121,5.5,5.9,3.6,10,60.0,rookie


In [149]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3443 entries, 0 to 37
Data columns (total 44 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   nombre_invocador    3443 non-null   object 
 1   lps                 3443 non-null   int64  
 2   ranking_mundial     3443 non-null   int64  
 3   ranking_porcentaje  3443 non-null   float64
 4   wins                3443 non-null   int64  
 5   loses               3443 non-null   int64  
 6   n_games             3443 non-null   int64  
 7   winrate_total       3443 non-null   float64
 8   liga_label          3443 non-null   int64  
 9   kills_player_kda    3443 non-null   float64
 10  deaths_player_kda   3443 non-null   float64
 11  assists_player_kda  3443 non-null   float64
 12  rol_1               3443 non-null   object 
 13  games_per_rol_1     3443 non-null   int64  
 14  winrate_per_rol_1   3443 non-null   float64
 15  champ_1             3443 non-null   object 
 16  regional

In [150]:
data = data.fillna('Unknown')

In [ ]:
# Etiquetamos median LaberEncoder los campeones que más utilizan nuestros jugadores para ser tomados como elementos
# del vector.

In [151]:
le=LabelEncoder()

for c in data.columns:
    if c != 'nombre_invocador' and c != 'rol_1' and c != 'nivel':
        if data.dtypes[c]==object:

            le.fit(data[c].astype(str))

            data[c]=le.transform(data[c].astype(str))

In [152]:
data.head()

,nombre_invocador,lps,ranking_mundial,ranking_porcentaje,wins,loses,n_games,winrate_total,liga_label,kills_player_kda,deaths_player_kda,assists_player_kda,rol_1,games_per_rol_1,winrate_per_rol_1,champ_1,regional_rank_1,kills_champ_1,deaths_champ_1,assists_champ_1,games_champ_1,winrate_champ_1,champ_2,regional_rank_2,kills_champ_2,deaths_champ_2,assists_champ_2,games_champ_2,winrate_champ_2,champ_3,regional_rank_3,kills_champ_3,deaths_champ_3,assists_champ_3,games_champ_3,winrate_champ_3,champ_4,regional_rank_4,kills_champ_4,deaths_champ_4,assists_champ_4,games_champ_4,winrate_champ_4,nivel
0,Agurin,1742,4,0.000043,242,174,436,57.6,2,6.4,4.6,8.5,Jungler,416,58.2,44,2,6.2,5.0,12.6,116,60.3,26,3641,7.2,3.8,6.7,100,67.0,144,42,5.9,4.3,7.0,68,55.9,61,3,6.3,5.1,7.8,51,51.0,rookie
1,6K OnFleek,1859,1,0.000010,150,111,260,57.7,2,6.3,4.7,7.3,Jungler,205,57.1,117,1,9.1,4.8,6.5,44,50.0,46,137,5.5,4.5,3.2,32,65.6,138,148,6.2,5.1,8.9,30,63.3,66,100,6.3,4.4,7.6,24,58.3,rookie
2,frosty,1760,3,0.000032,194,141,336,57.7,2,5.1,5.0,12.4,Support,249,57.4,9,1,3.6,4.9,13.5,68,52.9,53,21,4.2,4.5,16.8,34,64.7,41,70,4.6,5.4,11.6,34,58.8,95,37,2.0,4.1,17.4,33,60.6,rookie
3,Noah7,1782,2,0.000021,243,159,402,60.7,2,9.0,5.5,7.8,AD Carry,336,61.6,151,1,10.0,5.5,7.5,103,66.0,69,17,10.4,6.1,7.9,54,66.7,29,2,8.4,5.0,6.6,49,59.2,38,54,8.9,4.4,7.9,47,59.6,rookie
4,random gap,1056,315,0.003400,113,89,213,55.9,2,5.9,4.2,5.9,Mid,105,53.3,49,85,5.3,3.6,7.0,98,59.2,3,3745,8.9,5.1,5.2,29,55.2,125,7,6.7,4.4,4.1,11,45.5,47,121,5.5,5.9,3.6,10,60.0,rookie


In [ ]:
# Filtramos nuestro df para quedarnos con aquellos jugadores que juegan en el rol de jungla.

In [160]:
jun = data.loc[(data.rol_1 == 'Jungler')]

In [162]:
jun = jun.drop_duplicates()

In [163]:
jun.head()

,nombre_invocador,lps,ranking_mundial,ranking_porcentaje,wins,loses,n_games,winrate_total,liga_label,kills_player_kda,deaths_player_kda,assists_player_kda,rol_1,games_per_rol_1,winrate_per_rol_1,champ_1,regional_rank_1,kills_champ_1,deaths_champ_1,assists_champ_1,games_champ_1,winrate_champ_1,champ_2,regional_rank_2,kills_champ_2,deaths_champ_2,assists_champ_2,games_champ_2,winrate_champ_2,champ_3,regional_rank_3,kills_champ_3,deaths_champ_3,assists_champ_3,games_champ_3,winrate_champ_3,champ_4,regional_rank_4,kills_champ_4,deaths_champ_4,assists_champ_4,games_champ_4,winrate_champ_4,nivel
0,Agurin,1742,4,0.000043,242,174,436,57.6,2,6.4,4.6,8.5,Jungler,416,58.2,44,2,6.2,5.0,12.6,116,60.3,26,3641,7.2,3.8,6.7,100,67.0,144,42,5.9,4.3,7.0,68,55.9,61,3,6.3,5.1,7.8,51,51.0,rookie
1,6K OnFleek,1859,1,0.000010,150,111,260,57.7,2,6.3,4.7,7.3,Jungler,205,57.1,117,1,9.1,4.8,6.5,44,50.0,46,137,5.5,4.5,3.2,32,65.6,138,148,6.2,5.1,8.9,30,63.3,66,100,6.3,4.4,7.6,24,58.3,rookie
6,Techoteco v9,867,876,0.009400,152,126,319,56.1,2,6.0,5.2,8.8,Jungler,263,57.4,113,300,5.6,4.6,8.6,130,61.5,138,3582,6.9,5.3,9.5,77,57.1,152,5,4.4,4.5,9.9,51,51.0,142,53,9.8,6.9,6.8,9,44.4,rookie
9,Menchennik,846,949,0.010000,137,108,253,54.5,2,5.4,5.1,10.7,Jungler,220,55.0,136,281,6.1,5.8,11.4,40,57.5,75,147,3.7,4.7,13.3,39,69.2,105,168,5.5,3.6,13.1,38,52.6,147,125,6.5,5.5,8.0,22,59.1,rookie
11,Irode,846,950,0.010000,122,87,209,58.4,2,6.0,4.6,7.6,Jungler,188,59.0,25,283,7.2,3.9,7.5,116,65.5,106,5,3.2,4.7,9.2,16,50.0,138,1636,4.6,4.0,4.6,14,42.9,66,3779,5.9,7.4,10.4,11,45.5,rookie


In [ ]:
# Codificamos la columna de nivel

In [164]:
jun.nivel = jun.nivel.apply(lambda x: 1 if x == 'profesional' else 0)

In [ ]:
# Eliminamos la última columna categórica que nos queda

In [165]:
jun.drop('rol_1', axis = 1, inplace = True)

In [ ]:
# Implantamos como índice de nuestros df el nombre del jugador

In [166]:
jun = jun.set_index('nombre_invocador')

In [167]:
jun.head()

,lps,ranking_mundial,ranking_porcentaje,wins,loses,n_games,winrate_total,liga_label,kills_player_kda,deaths_player_kda,assists_player_kda,games_per_rol_1,winrate_per_rol_1,champ_1,regional_rank_1,kills_champ_1,deaths_champ_1,assists_champ_1,games_champ_1,winrate_champ_1,champ_2,regional_rank_2,kills_champ_2,deaths_champ_2,assists_champ_2,games_champ_2,winrate_champ_2,champ_3,regional_rank_3,kills_champ_3,deaths_champ_3,assists_champ_3,games_champ_3,winrate_champ_3,champ_4,regional_rank_4,kills_champ_4,deaths_champ_4,assists_champ_4,games_champ_4,winrate_champ_4,nivel
nombre_invocador,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Agurin,1742,4,0.000043,242,174,436,57.6,2,6.4,4.6,8.5,416,58.2,44,2,6.2,5.0,12.6,116,60.3,26,3641,7.2,3.8,6.7,100,67.0,144,42,5.9,4.3,7.0,68,55.9,61,3,6.3,5.1,7.8,51,51.0,0
6K OnFleek,1859,1,0.000010,150,111,260,57.7,2,6.3,4.7,7.3,205,57.1,117,1,9.1,4.8,6.5,44,50.0,46,137,5.5,4.5,3.2,32,65.6,138,148,6.2,5.1,8.9,30,63.3,66,100,6.3,4.4,7.6,24,58.3,0
Techoteco v9,867,876,0.009400,152,126,319,56.1,2,6.0,5.2,8.8,263,57.4,113,300,5.6,4.6,8.6,130,61.5,138,3582,6.9,5.3,9.5,77,57.1,152,5,4.4,4.5,9.9,51,51.0,142,53,9.8,6.9,6.8,9,44.4,0
Menchennik,846,949,0.010000,137,108,253,54.5,2,5.4,5.1,10.7,220,55.0,136,281,6.1,5.8,11.4,40,57.5,75,147,3.7,4.7,13.3,39,69.2,105,168,5.5,3.6,13.1,38,52.6,147,125,6.5,5.5,8.0,22,59.1,0
Irode,846,950,0.010000,122,87,209,58.4,2,6.0,4.6,7.6,188,59.0,25,283,7.2,3.9,7.5,116,65.5,106,5,3.2,4.7,9.2,16,50.0,138,1636,4.6,4.0,4.6,14,42.9,66,3779,5.9,7.4,10.4,11,45.5,0


In [ ]:
# En este punto, tenemos un df que tiene como indices el nombre del jugador. El cuerpo de nuestra tabla son todas
# las características que se miden de manera numérica. Estos números conforman un vector y 
# por lo tanto una posición en el plano. 
# El siguiente paso será calcular la distancia de cada uno de nuestros vectores con respecto del resto.


In [169]:
distancias = squareform(pdist(jun_trans, 'euclidean'))    

distancias[0].shape

(717,)

In [180]:
distancias

array([[   0.        , 3523.96735796, 1302.53499381, ..., 3737.77113666,
        4965.81682304, 4886.15893529],
       [3523.96735796,    0.        , 3709.72006627, ..., 1327.95811307,
        3363.90539998, 3335.45409516],
       [1302.53499381, 3709.72006627,    0.        , ..., 3524.6099841 ,
        4823.03247035, 4243.48445861],
       ...,
       [3737.77113666, 1327.95811307, 3524.6099841 , ...,    0.        ,
        3294.66241822, 2524.40508843],
       [4965.81682304, 3363.90539998, 4823.03247035, ..., 3294.66241822,
           0.        , 3144.83278103],
       [4886.15893529, 3335.45409516, 4243.48445861, ..., 2524.40508843,
        3144.83278103,    0.        ]])

In [ ]:
# Una vez conseguimos una matriz con las distancias entre cada vector, utilizamos estos valores para calcular la
# similitud entre cada uno de ellos. Este es el valor más valioso para la consecución del objetivo del presente
# proyecto.

In [170]:
similitud = 1 / (1 + distancias)

similitud.shape

(717, 717)

In [181]:
similitud

array([[1.00000000e+00, 2.83690570e-04, 7.67144729e-04, ...,
        2.67467562e-04, 2.01336195e-04, 2.04617859e-04],
       [2.83690570e-04, 1.00000000e+00, 2.69489474e-04, ...,
        7.52469164e-04, 2.97185175e-04, 2.99719394e-04],
       [7.67144729e-04, 2.69489474e-04, 1.00000000e+00, ...,
        2.83638861e-04, 2.07295454e-04, 2.35599873e-04],
       ...,
       [2.67467562e-04, 7.52469164e-04, 2.83638861e-04, ...,
        1.00000000e+00, 3.03429136e-04, 3.95976077e-04],
       [2.01336195e-04, 2.97185175e-04, 2.07295454e-04, ...,
        3.03429136e-04, 1.00000000e+00, 3.17880851e-04],
       [2.04617859e-04, 2.99719394e-04, 2.35599873e-04, ...,
        3.95976077e-04, 3.17880851e-04, 1.00000000e+00]])

In [ ]:
# Transformamos nuestra matriz de similitud en un DataFrame, el cual contendrá la relación de similitud entre 
# todos nuestros jugadores y que por tanto, nos permitirá conocer cuales son los jugadores rookies que más se 
# parecen a los jugadores profesionales con los que los estamos comparando.

In [171]:
jun_sim = pd.DataFrame(similitud, index=jun.index, columns=jun.index)

jun_sim.head()

nombre_invocador,Agurin,6K OnFleek,Techoteco v9,Menchennik,Irode,King,UDYSOF,H0NE,Eri my beloved,피그말랑 효과,Paresz14,kanaaa,oerwoud,Tarryasepin,Billy Butcher XD,Billy Butcher XD,Billy Butcher XD,Billy Butcher XD,Billy Butcher XD,Billy Butcher XD,Billy Butcher XD,Billy Butcher XD,Kosanuemi,Zelzeleniş,ASTROBOY99,where is shee,FA Vinite,Mimage,a lot 3rd verse,TETIS XDDDDD,PAGYS,zyb,STEPZ,푸키키키키키,FUT Mojito,giveupp,zuao og,im a jungler,JUGKlNG,Badlulu fan,erevin,zikiinha,STP ME PARLE PAS,職業玳打微ysrw66,im 16 years old,DuplaNazareno,Bo Fan,MENTAL TXRRORlSM,112358132134558,BrotherGaren,教學 dc迎神6305,Michi Wo Yuke,Thaybot,Canon,LeaningTable TTV,kenvi9,但偏偏雨漸漸,LBR Drakehero,Ed Rebirth,ITSJUSTAGAMEDUDE,Boogeyman,Gragolandia,再见,RED Curse,tkffuwnj,ssywfnkhl,paiN anato,Negań Smith,lNote,Śheiden,Multirole Talent,Køldo Locuraa,KJ Allen,jukes fan,eightQAQ,Perfect Blue,Draven696969,100T C1oser,AERO X BANE PAIN,JhompysJr,ICrazzy Ekko,BUFF KEQING,Soweto,Karina167,Funny Youu,모두 행복해요,twtchtv detdert,林小希,Hanisito,Wx Mjm978244659,BT1 Levi,TH Krillin,nemesis fan 48,Yagiao,Ozilardann,IW Elramir,Albetrayber,VELJA THE MENACE,Sylvie,NattyNatt,Trixter2,KeeI,sarolu,dw99q0znyh9c7wh,Sarcasm1,니가 깨기 전에,MorninStar,LEWANDOWSKI 9,HLE Key1,MrGraek,hwa i tings,Lowest,lalms,Juhan,Charolon,앙 맛있엉,Aniayua,RhiIech,LF BLACK,mxxxx,all dog,Kneking,LOUD Ronaldinho,WillowTheRaccoon,YNW MELLYY,Curselol1,jamican banana,Cesaroxx,Big Genos,no more slacking,paiN caca1,BABO JUNGLE1,Luccho,Yøø Høbin,Zwei und die,주사는 무서워,The 글로리,Villager C,Xinxa7,FORA999,Pentaless,readysetgo1,陪daiVX Ekko4567,Yuu13,Happy Alexx,gRIZZlybear123,JUGKING,Matty,VIV Joinze,WillNA1,Leemo,PCS Icarus,thug ward,Boom2G,Shortley,Dani Marshall,Yogur de Mango,da go lapa6,WAP 5,Promise to Dark,APOJHE,Ai Hayasaka,TwTvLeaningTable,sensitive3,NNJ Azrawr,EmbraceLife,LOS CrjaY,Ice Prince,Sybol,Оutsidеr,exselor,Josedeodo,Fanatiik,naiyoubaby,Gilius2,타 잔,咩啊咩啊,Astrax00,KIREAS1,Timon,DK Lucid,kisno,AeryIsKawaii,flash2468,NNOne Piece,HamBak,AHaHaCiK,KOI Malrang1,Bow,Yuxiayizhengwan,PB Lord Semi,陪袋V xinshenjug,TaiYangShen,lalalalalalaaala,Not Hans,MartinSON,maimai0v0,Brother Gwang,eilataN,DaBox,KANG HYUNJUN,不滅巔峰,Renascence,JUGKlNG1,EMlNATR1X,R0CK Gucio,qweasdcxz,Katten,EmbraceYourPain,Music101,redcrab84,我女友還沒16歲,KeyNG,B0NN1E,Nightmisty,719,Now,aesthetic addict,MANATY RANK 1,갱스기무라,jgWorm,Diva Thủ Dầu 1,qqxasssdd,Ankhsëräm,Curling Captain,Jãger,dlwlrma fan,英短金漸層我的貓,gggman,kidbuuprovip,Glorias,IIiiiillllllIII,DWG 암Canyon,Oscaarin,Piwo,HLE Grizzly,Chaménos,totally sane,cutexin1,FromScz,野區上等馬,Izahn,Izahn,H1ng,No one for u,SHAŁŁ,祭改你公,Mathý,SPACEX ASTRONAUT,GZeus,º X º,溫言莞爾º,dantes enjoyer,wx 1065620772,일산전영찬,관 계,void rüsh,void rüsh,Numvu,HêrãclësG,HêrãclësG,恩於南歌0o,Qiiqiii,Remax201,KKayn,Basilekun,Hachi,Minh Inox,24 ËKÎM 2015,iLikeThickThighs,senzawa,Agoniº,SchuJerer Gustav,L9 Myκys,Çhích ßôÑg,she left 4 chad,clones i gibones,Death Sigh,zacocaca ívar,Aeolus,eIo terrorıst,Neo ZeTHRIV,我不爛了,Buffy,Damngøre,Sileñt Kîllèr,Íta,Fear,bialy mbappé,Jayder,1223334444,OhSehun1,神奇的丸小丸,ShrimpVicious,Disco Goguś,kitasanb,すや姫,Sword Skill,FEAR OF DEAD,PRlDE,bun ngu,清冠123,Run Higher,UnExpecteDGanGS,Happy Game 0923,玩不了那就点,野區爸爸,ExpressKill1043,jg gap 15open,Mbappé2025,Pochitx,fmr,그만 날 놔줘,聖 經,Void Is So Empty,Agiłity,Chaeyeoni stan,l1ttlefish,The Speed Freak,Saabaa,講話會分心,雪花若離乃願折翼,6I Ni,Ðegurechaff,Naethancolch,Kaneki,MΟGLIS,blueblue,KuLLeR972,LEV Nikolaic,Ụt Ụt,哆垃貢,swaglord,talon2,wutian,Dexti,Pb Lead,UnFear,寒塘冷月,KatieÇ,어흥 놀랐찌,TwTv Crawlxzz,Microvolts TCL,Scrappy,Maasta,heâven,Permaban07845219,Shysaßßòüó,Shysaßßòüó,TryhardEkko,Last summer,batıya gidiyom,等等吃鹹水雞,Beyond Souls,Sayın Bezmenlerr,Paradizzo,SewerBoy,HGM IBrunoide,LookLikeGod,UANL Gio,Wsavh,Shoujae,1Fresco,츄개잘해서미안해,Die4U,desti,ATC Sonechsko,IIlllIIlllII,Gabriela Infante,DilysQuang,Gwangdae,4 9 5,Has God Complex,愛と誠,普通玩家,Azunyan,H O À N G T Â M1,ワニノコ,yбей мeня,Script I feed,xxxxlululu,Lost,Adequate,Gragascı Baki,mâHSEN,LORASOTGAY69,The Devil Withiñ,年近立年老打野,Elokrix Fanboy,Sorien,LER Norar,just dont care,Kurau

In [172]:
jun_sim.shape

(717, 717)

In [64]:
jun_rook = data.nombre_invocador.loc[(data.rol_1 == 'Jungler') & (data.nivel != 'profesional')]

In [ ]:
# Eliminamos de nuestras columnas aquellas que se correspoden con los jugadores rookies, de manera que solo quede
# la comparativa con respecto a los jugadores profesionales.

In [173]:
col = jun_sim.columns

In [174]:
col[:-8]

Index(['Agurin', '6K OnFleek', 'Techoteco v9', 'Menchennik', 'Irode', 'King',
       'UDYSOF', 'H0NE', 'Eri my beloved', '피그말랑 효과',
       ...
       'Maxxxxxxxx', 'wenwinwenwinwenw', '반오십의 희망', 'Ziix Garguentua',
       'REALG4LlFE', 'REALG4LlFE', '9드론스포닝풀발업저글링', '9드론스포닝풀발업저글링',
       'Wynncraftian', 'XiaoHuaSheng7'],
      dtype='object', name='nombre_invocador', length=709)

In [175]:
jun_sim.drop(col[:-8], axis = 1, inplace = True)

In [176]:
jun_sim.head()

nombre_invocador,elyoyaaaaaaaaaaa,Razørk Activoo,잘가요 굿바이,blaberfish2,I Love Uber Eats,Maxlore,Boukada,bluerzor
nombre_invocador,,,,,,,,
Agurin,0.000215,0.000258,0.000187,0.000291,0.000277,0.000267,0.000201,0.000205
6K OnFleek,0.000308,0.000750,0.000140,0.001613,0.001440,0.000752,0.000297,0.000300
Techoteco v9,0.000249,0.000266,0.000214,0.000285,0.000276,0.000284,0.000207,0.000236
Menchennik,0.000456,0.000881,0.000152,0.000814,0.000975,0.001730,0.000309,0.000437
Irode,0.000292,0.000322,0.000141,0.000267,0.000251,0.000247,0.000655,0.000294


In [ ]:
# Calculamos nuestro score sumando todas las tasas de similitud de los jugadores rookies con respecto a 
# los profesionales. De esta manera tendremos una aproximación de aquellos jugadores que tienen potencial para 
# ser fichados por equipos profesionales.

In [177]:
jun_sim['total'] = jun_sim.sum(axis = 1)

In [182]:
jun_mando = jun_sim.sort_values(by='total', ascending = False)

In [183]:
jun_mando.head()

nombre_invocador,elyoyaaaaaaaaaaa,Razørk Activoo,잘가요 굿바이,blaberfish2,I Love Uber Eats,Maxlore,Boukada,bluerzor,total
nombre_invocador,,,,,,,,,
elyoyaaaaaaaaaaa,1.000000,0.000403,0.000169,0.000334,0.000339,0.000412,0.000317,0.003645,1.005621
bluerzor,0.003645,0.000393,0.000164,0.000324,0.000329,0.000396,0.000318,1.000000,1.005569
I Love Uber Eats,0.000339,0.001096,0.000140,0.001714,1.000000,0.001209,0.000324,0.000329,1.005152
blaberfish2,0.000334,0.001162,0.000145,1.000000,0.001714,0.000818,0.000343,0.000324,1.004840
Razørk Activoo,0.000403,1.000000,0.000144,0.001162,0.001096,0.000867,0.000437,0.000393,1.004503


In [ ]:
# Exportamos nuestro df definitivo 

In [184]:
jun_mando.to_csv('../data/scores/jun_score.csv')